Importing Libraries

In [2]:
from tkinter import *
import tkinter as tk
from keras.models import load_model
import tkinter.ttk as ttk
import time
import cv2
import shutil
import numpy as np
from function import *
from tkinter.font import Font
import os
import serial
from PIL import ImageTk, Image

In [3]:

model_detect = load_model('Papaya_Detection_Model.h5')
model_disease = load_model('aug_model.h5')


window for splash screen (loading screen when processing a papaya)

In [4]:

def splash():

    #Creates window
    global cam
    w=Tk()
    width_of_window = 427
    height_of_window = 250
    screen_width = w.winfo_screenwidth()
    screen_height = w.winfo_screenheight()
    x_coordinate = (screen_width/2)-(width_of_window/2)
    y_coordinate = (screen_height/2)-(height_of_window/2)
    w.geometry("%dx%d+%d+%d" %(width_of_window,height_of_window,x_coordinate,y_coordinate))
    w.overrideredirect(1)
    w.wm_attributes("-topmost", 1)


    s = ttk.Style()
    s.theme_use('clam')
    s.configure("red.Horizontal.TProgressbar", foreground='red', background='#4f4f4f')
    progress=ttk.Progressbar(w,style="red.Horizontal.TProgressbar",orient=HORIZONTAL,length=500,mode='determinate',)

    def stop_buttons():
        print("Program is Stoping")
        w.destroy()

    #############progressbar          33333333333333333333333333333
    def new_win():
        w.destroy()
        q=Tk()
        q.title('Main window')
        q.geometry('427x250')
        l1=Label(q,text='ADD TEXT HERE ',fg='grey',bg=None)
        l=('Calibri (Body)',24,'bold')
        l1.config(font=l)
        l1.place(x=80,y=100)
        
        
        
        q.mainloop()



    def bar(w,cam):
        
        global r
        l4=Label(w,text='Loading...',fg='white',bg=a)
        lst4=('Calibri (Body)',10)
        l4.config(font=lst4)
        l4.place(x=18,y=210)
        img = cv2

        while True:
            ret, frame = cam.read()

            #Resize the frame(image captured) to 224x224
            frame = cv2.resize(frame, (224,224))
            #assign fram ti
            disease_img = frame
            frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)

            papaya = detect_papaya(disease_img)
        

            if papaya:
                r=0
                progress['value']=r
                w.update_idletasks()
                time.sleep(0.03)
                l4.config(text='Taking Photos',fg='white',bg=a)
            
                
                r=r+10
                progress['value']=r
                w.update_idletasks()
                main(w,progress,l4)
                time.sleep(0.03)
                r=r+10

                progress['value']=r
                w.update_idletasks()
                time.sleep(0.03)
                l4.config(text='Processing Photos',fg='white',bg=a)
            

                progress['value']=r
                w.update_idletasks()
                Maturity,disease_list_final = process(w,progress,l4)
                time.sleep(0.03)
                r=r+10

                progress['value']=r
                w.update_idletasks()
                time.sleep(0.03)
                r=r+10
                weight = 1030
                progress['value']=r
                w.update_idletasks()
                time.sleep(0.03)
                r=r+10
                progress['value']=r
                w.update_idletasks()
                time.sleep(0.03)
                r=r+10
                progress['value']=r
                w.update_idletasks()
                time.sleep(0.03)
                r=r+10
                """
                progress['value']=r
                w.update_idletasks()
                time.sleep(0.03)
                r=r+10
                progress['value']=r
                w.update_idletasks()
                time.sleep(0.03)
                r=r+10
                progress['value']=r
                w.update_idletasks()
                time.sleep(0.03)
                r=r+10
                """
                start_prog(w,disease_list_final,Maturity,weight)

            else:
                no_papaya(w,cam)
            
                
        
    progress.place(x=-10,y=235)

    a='#249794'
    x = Frame(w,width=427,height=241,bg=a)
    x.place(x=0,y=0)  #249794
    b1=Button(w,width=10,height=1,text='START',command=lambda :bar(w,cam),border=0,fg=a,bg='white')
    b1.place(x=170,y=200)

    stop = tk.Button(w,text='Stop', padx= 20,command=stop_buttons)
    stop.grid(sticky=tk.NE)

    l1=Label(w,text='Processing',fg='white',bg=a)
    lst1=('Calibri (Body)',18,'bold')
    l1.config(font=lst1)
    l1.place(x=50,y=80)

    l2=Label(w,text='Papaya',fg='white',bg=a)
    lst2=('Calibri (Body)',18)
    l2.config(font=lst2)
    l2.place(x=185,y=82)

    l3=Label(w,text='Papaya Project',fg='white',bg=a)
    lst3=('Calibri (Body)',10)
    l3.config(font=lst3)
    l3.place(x=50,y=110)

    


    w.mainloop()


In [5]:
def process(w,progress,l4):
    #w,progress,l4
    global model_disease,a,r,dirname2
    count = 0
    dirname = dirname2
    colorlist = []
    
    disease_list_final=  {'anthracnose':0,'black_spot':0,'phytophthora':0,'powdery_mildew':0,'ring_spot':0}
    percentage_list = []
    green_sum = 0
    yellow_sum = 0
    ran = 2

    for i in range (ran):
        
        print('######################################################')
        print('             Image               ' + str(i))
        
        path = os.path.join(dirname,  "papaya_images_{}".format(str(i) + '.png'))
        print(path)
        cam = cv2.imread(path)
        cv2.imwrite('image.png',cam)
        frame = cv2.resize(cam, (224,224))
        frame1 = np.array(frame)
        disease_img = frame
        maturity_img = frame1

        
        cv2.imwrite("PapayaImage.png", disease_img)

        disease_list= output(disease_img, model_disease)
        
        #show(frame, disease, percentage, maturity[0])
        green, yellow = detect_maturity(maturity_img)
        
        print("Image ",i, " Green: ", green, " Yellow: ", yellow)
        colorlist.append((green,yellow))

        green_sum += green
        yellow_sum += yellow
        
        for key in disease_list_final:
            if disease_list[key] > disease_list_final[key]:
                disease_list_final[key] = disease_list[key]

        print('######################################################')
        
        """
        l4.config(text="image '"+ str(i+1) + "'of 4............",fg='white',bg=a)
      
        

        time.sleep(0.03)
        r=r+10
        progress['value']=r
        w.update_idletasks()
        """
        


        
    print("Disease list = " ,disease_list_final)
    print("G/Y total = " ,green_sum,yellow_sum)
    print("Ave Green = ", green_sum/ran)
    print("Ave Yellow = ", yellow_sum/ran)
    """
    for i in range(len(colorlist)):
        print("Image" , i , "=", colorlist[i])
        print(colorlist[i][0],colorlist[i][2])
        green_sum += colorlist[i][0]
        yellow_sum += colorlist[i][2]
    """
    

    


    green = green_sum/ran
    yellow = yellow_sum/ran
    print("Diseases : ", disease_list,percentage_list)
    print("Green: ", green)
    print("Yellow: ",yellow)


    if (green > 90) & (yellow <10):
        Maturity = "green"
    elif (green > 80) & (yellow < 20):
        Maturity = "breaker"
    elif (green > 60) & (yellow < 30):
        Maturity = "1/3 ripe"
    elif (green >= 50) or (yellow <=50):
        Maturity ="half ripe"
    elif (green <= 20) & (yellow > 80):
        Maturity = "ripe"
    else:
        Maturity =  "table ripe"

    print("Maturity = " ,Maturity)
    return (Maturity,disease_list_final)
        

In [6]:
def no_papaya(w,cam):
    w.destroy()


    w=Tk()


    width_of_window = 427
    height_of_window = 250
    screen_width = w.winfo_screenwidth()
    screen_height = w.winfo_screenheight()
    x_coordinate = (screen_width/2)-(width_of_window/2)
    y_coordinate = (screen_height/2)-(height_of_window/2)
    w.geometry("%dx%d+%d+%d" %(width_of_window,height_of_window,x_coordinate,y_coordinate))


    w.overrideredirect(1)


    s = ttk.Style()
    s.theme_use('clam')
    s.configure("red.Horizontal.TProgressbar", foreground='red', background='#4f4f4f')
    progress=ttk.Progressbar(w,style="red.Horizontal.TProgressbar",orient=HORIZONTAL,length=500,mode='determinate',)

    def stop_buttons(w):
        print("Program is Stoping")
        w.destroy()
    def restart_buttons(w):
        print("Program is Restarting")
        w.destroy()
        splash()

    #############progressbar          33333333333333333333333333333
    def new_win():
    # w.destroy()
        q=Tk()
        q.title('Main window')
        q.geometry('427x250')
        l1=Label(q,text='ADD TEXT HERE ',fg='grey',bg=None)
        l=('Calibri (Body)',24,'bold')
        l1.config(font=l)
        l1.place(x=80,y=100)
        
        
        
        q.mainloop()



    def bar(w,cam):
        global r
        l4=Label(w,text='Loading...',fg='white',bg=a)
        lst4=('Calibri (Body)',10)
        l4.config(font=lst4)
        l4.place(x=18,y=210)
        
        papaya = detect_papaya()

        if papaya:
            splash()
        else:
            no_papaya(w,cam)
        

        
            
                
        
    progress.place(x=-10,y=235)




    #############
    # frame 333333333333333333333333
    #
    ###########





    '''

    def rgb(r):
        return "#%02x%02x%02x" % r
    #Frame(w,width=432,height=241,bg=rgb((100,100,100))).
    '''
    a='#E97451'
    x = Frame(w,width=427,height=241,bg="#E97451")
    x.place(x=0,y=0)  #249794
    b1=Button(w,width=10,height=1,text='RESTART',command=lambda :restart_buttons(w),border=0,fg=a,bg='white')
    b1.place(x=120,y=200)

    b2=Button(w,width=10,height=1,text='STOP',command=lambda :stop_buttons(w),border=0,fg=a,bg='white')
    b2.place(x=200,y=200)
    ######## Label

    l1=Label(w,text='No Papaya',fg='white',bg=a)
    lst1=('Calibri (Body)',18,'bold')
    l1.config(font=lst1)
    l1.place(x=50,y=80)

    l2=Label(w,text='Detected',fg='white',bg=a)
    lst2=('Calibri (Body)',18)
    l2.config(font=lst2)
    l2.place(x=185,y=82)

    l3=Label(w,text='Papaya Project',fg='white',bg=a)
    lst3=('Calibri (Body)',10)
    l3.config(font=lst3)
    l3.place(x=50,y=110)

    


    w.mainloop()



In [7]:
def main_program():
    global img_counter, dirname1,Arduino
    

    #Delete existing directory 'test'
    shutil.rmtree(dirname1)

    #Creates directory 'test' 
    os.mkdir(dirname1)

    img_counter = 0
    #Arduino.write(b'E')


    #Starts taking pictures
    while True:
        ret, frame = cam.read()

        #Resize the frame(image captured) to 224x224
        frame = cv2.resize(frame, (224,224))
        #assign fram ti
        disease_img = frame
        frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        

        capture(frame)
        
        frame1 = np.array(frame)
        maturity_img = cv2.cvtColor(frame1,cv2.COLOR_BGR2RGB)

        #confirm_papaya = (detect_papaya(disease_img,model_detect))
        #print('CONFIRM PAPAYA: ', confirm_papaya)


        img_list = []
        roll()
        time.sleep(1)

        cv2.imwrite("PapayaImage.png", disease_img)
        if img_counter>3:
            Arduino.write(b'D')
            cam.release()
            break


In [8]:
def capture(img):
        global img_counter, dirname1
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img_name = "papaya_images_{}.png".format(img_counter)
        cv2.imwrite(os.path.join(dirname1, img_name), img)
        print("Image Taken ", [img_counter])
        img_counter += 1

In [9]:
def start_prog2(main,disease_list_final,maturity_detected,weight):
    #main, model_detect, model_disease,disease_list_final,maturity_detected,weight
    #main, model_detect, model_disease,disease_list_final,maturity,weight
    print('Loading start_prog..')
    global cam
    """"""
    main.destroy()
  
    folder = r'test2'

    #weight = 100
    maturity_detected = "Table Ripe"
    #disease_list_final=  {'anthracnose':0,'black_spot':0,'phytophthora':0,'powdery_mildew':0,'ring_spot':0}
    
    #disease = "antracnose"
    path = os.listdir(folder)
    print(path)
    newpath = os.path.join(folder,(path[index1]))
    print(newpath)

    """
    def update_weight():
        Arduino.write(b'R')
        weight = Arduino.readline().decode('utf-8').rstrip()
    
    update_weight()
    """
    
    
    main = tk.Tk()
    width_of_window = 480
    height_of_window = 320
    screen_width = main.winfo_screenwidth()
    screen_height = main.winfo_screenheight()
    x_coordinate = (screen_width/2)-(width_of_window/2)
    y_coordinate = (screen_height/2)-(height_of_window/2)
    main.geometry("%dx%d+%d+%d" %(width_of_window,height_of_window,x_coordinate,y_coordinate))

    main.configure(background='black')
    main.title('Papaya Project')
    main.resizable(width = False, height= False)
    main.overrideredirect(True)
    main.wm_attributes("-topmost", 1)
    

    def restart_buttons(main):
        main.destroy()
        splash()

    def stop_buttons(main):
        main.destroy()
        

    header = tk.Frame(main,width=480, height = 40, bg = '#009ea0',bd = 0)
    header.grid(row = 0, columnspan=6 , column = 0)
    
    restart = tk.Button(main,text='Restart',command=lambda: restart_buttons(main), padx= 20)
    restart.grid(row = 0, column = 3 ,sticky=tk.W)

    stop = tk.Button(main,text='Stop', padx= 20,command=lambda : stop_buttons(main))
    stop.grid(row = 0, column = 4,columnspan=2 ,sticky=tk.W)

    body = tk.Frame(main,width=480, height = 250, bg = 'lightblue',bd = 0)
    body.grid(row = 1,column = 0, columnspan= 6, rowspan=4)

    imageframe = tk.Frame (main, width= 260, height= 250, bg = 'white' )
    imageframe.grid(row = 1, column=0, sticky= tk.W, columnspan=3, rowspan =4 )

    #outputframe = Frame(s, width=220, height=250, bg = 'red')
    #outputframe.grid(row = 1, rowspan=4, column=3, columnspan=3)

    output = tk.Label(main,text = 'Output',bg = 'lightblue',fg = 'black',bd = 0 , pady=10, font =('Times',15,'bold'))
    output.grid(row = 1, columnspan= 3, rowspan=2, column= 3, sticky = tk.N)
    #output.place(x = 300, y = 280)

    maturity_text = tk.Label(main, text ='Maturity Level : ', bg = 'lightblue' ,fg = 'black',pady=10, font =('Times',10))
    maturity_text.grid(row = 2, column= 3 , columnspan= 2, sticky=tk.W )

    

    maturity_bg= tk.Frame(main, width=60, height= 40, bg = '#333333')
    maturity_bg.grid(row = 2, column= 4 , columnspan= 3, sticky=tk.W )
    

    maturity = tk.Label(maturity_bg, text =maturity_detected, bg = '#DDDDDD' ,fg = '#222222',pady=10, font =('Times',10))
    maturity.grid(row = 2, column= 4 , columnspan= 2, sticky=tk.W )

    disease_text = tk.Label(main, text ='Disease Detected : ', bg = 'lightblue' ,fg = 'black',pady=10)
    #disease_text.grid(row = 3, column= 3 , columnspan= 2, sticky=tk.NW )
    #disease_text = tk.Label(main, text="\n".join([f"{key}: {value}" for key, value in disease_list_final.items()]))
    disease_text.grid(row = 3, column= 3 , columnspan= 2, sticky=tk.NW )

    disease_bg= tk.Frame(main, width=60, height= 40, bg = '#333333')
    disease_bg.grid(row = 3, column= 4 , columnspan= 3, sticky=tk.NW )

    disease = tk.Label(main, text="\n".join([f"{key}: {value}" for key, value in disease_list_final.items()]))
    disease.grid(row = 3, column= 4 , columnspan= 2, sticky=tk.NW )
    

    size_text = tk.Label(main, text ='Weight : ', bg = 'lightblue' ,fg = 'black',pady=10)
    size_text.grid(row = 4, column= 3 , columnspan= 2, sticky=tk.NW )

    size_bg= tk.Frame(main, width=60, height= 40, bg = '#333333')
    size_bg.grid(row = 4, column= 4 , columnspan= 3, sticky=tk.NW )

    size = tk.Label(size_bg, text =str(weight) + 'grams' , bg = '#DDDDDD' ,fg = '#222222',pady=10, font =('Times',10))
    size.grid(row = 4, column= 4 , columnspan= 2, sticky=tk.NW )

    #body.place(x = 0, y = 50)
    
    footer = tk.Frame(main,width=480, height = 35, bg = 'white',bd = 0)
    footer.grid(row = 5,column=0 ,columnspan= 6, rowspan=3, sticky = tk.N)

    
    
    
    
    #navigate = Frame(s,width=150, height=20,bg = 'blue',bd = 0 )
    #navigate.grid(row = 6, columnspan= 3, column= 0, sticky= NW,padx = 20)
    #navigate.place(x = 65 ,y = 300)
    """
    logo = Image.open(r'papaya_logo.png')
    logo = logo.resize((40,30))
    logo = ImageTk.PhotoImage(logo)
    image_label = tk.Label(main,image=logo, bd = 0,bg = 'lightblue')
    image_label.image = logo
    image_label.grid(row = 0, column = 0,columnspan=4 ,sticky=tk.E)
    image_label.place(x = 5, y = 5)
    """
   
    title = tk.Label(main, text  = 'Papaya \nProject', bd = 0, fg ='#343434'  ,
                font=('sans',9,'bold'), justify = 'left', bg = '#009ea0', pady=5)
    title.grid(row = 0, column=1, columnspan=5)
    title.place(x = 60)

    """
    
    path = os.listdir(folder)
    print(path)
    newpath = os.path.join(folder,(path[index1]))
    print(newpath)"""
    
    img = Image.open(newpath)
    img = img.resize((244,244))
    img = ImageTk.PhotoImage(img)
    imglabel = tk.Label(imageframe,image = img)
    imglabel.image = img
    imglabel.grid(row = 1, column=0, sticky= tk.W, columnspan=3, rowspan =4)
    #imglabel.place(x= 5, y = 55)
   
    

    def rightbutton(newpath,path,img, imglabel):
        global index1
        if index1 < len(path) - 1:
            index1 += 1

        print(index1)
        newpath  = os.path.join(folder,(path[index1]))
        img = Image.open(newpath)
        img = img.resize((244,244))
        img = ImageTk.PhotoImage(img)
        imglabel = tk.Label(main,image = img)
        imglabel.image = img
        imglabel.grid(row = 1, column=0, sticky= tk.W, columnspan=3, rowspan =4)

        items = tk.Label(main, text = 'Image %i of %i' %(index1+1, len(path)), padx = 10)
        items.grid(row=6, column=1)

    def leftbutton(newpath,path,img, imglabel):
        global index1
        
        if index1 > 0:
            index1 -= 1

        print(index1)
        newpath  = os.path.join(folder,(path[index1]))
        
        print(newpath)
        imglabel.grid_forget()
        
        img = Image.open(newpath)
        img = img.resize((244,244))
        img = ImageTk.PhotoImage(img)
        imglabel = tk.Label(main,image = img)
        imglabel.image = img
        imglabel.grid(row = 1, column=0, sticky= tk.W, columnspan=3, rowspan =4)

        items = tk.Label(main, text = 'Image %i of %i' %(index1+1, len(path)), padx = 10)
        items.grid(row=6, column=1)




    right = tk.Button(main, text = ">", command = lambda : rightbutton(newpath,path,img, imglabel),padx = 10 , bg ='#009ea0')
    left = tk.Button(main, text = "<", command = lambda : leftbutton(newpath,path,img, imglabel),padx = 10, bg ='#009ea0' )

    right.grid(row = 6, column = 2 ,sticky=tk.NW)
    #right.place(y = 258, x= 410 )
    left.grid(row = 6, column=0, sticky=tk.NE)
    #left.place(y = 258, x= 300 )

    items = tk.Label(main, text = 'Image %i of %i' %(index1+1, len(path)), padx = 10)
    items.grid(row=6, column=1)


    
  
    main.mainloop()

In [10]:
def start_prog(main,disease_list_final,maturity_detected,weight):
    #main, model_detect, model_disease,disease_list_final,maturity_detected,weight
    #main, model_detect, model_disease,disease_list_final,maturity,weight
    print('Loading start_prog..')
    global dirname2
    """"""
    main.destroy()
  
    folder = dirname2

    #weight = 100
    maturity_detected = "Table Ripe"
    #disease_list_final=  {'anthracnose':0,'black_spot':0,'phytophthora':0,'powdery_mildew':0,'ring_spot':0}
    
    #disease = "antracnose"
    path = os.listdir(folder)
    print(path)
    newpath = os.path.join(folder,(path[index1]))
    print(newpath)

    """
    def update_weight():
        Arduino.write(b'R')
        weight = Arduino.readline().decode('utf-8').rstrip()
    
    update_weight()
    """
    max_value = max(disease_list_final.values())
    max_keys = [k for k, v in disease_list_final.items() if v == max_value]
    for key in max_keys:
        if disease_list_final[key]>95:
            highest = max_keys
        else:
            highest = "No_Disease"
    
    
    
    main = tk.Tk()
    width_of_window = 427*1.5
    height_of_window = 250*1.5
    screen_width = main.winfo_screenwidth()
    screen_height = main.winfo_screenheight()
    x_coordinate = (screen_width/2)-(width_of_window/2)
    y_coordinate = (screen_height/2)-(height_of_window/2)
    main.geometry("%dx%d+%d+%d" %(width_of_window,height_of_window,x_coordinate,y_coordinate))

    main.configure(background='black')
    main.title('Papaya Project')
    main.resizable(width = False, height= False)
    #main.overrideredirect(True)
    main.wm_attributes("-topmost", 1)
    
    roboto_font = Font(family="Roboto", size=10)

    def restart_buttons(main):
        main.destroy()
        splash()

    def stop_buttons(main):
        main.destroy()
        

    header = tk.Frame(main,width=480, height = 40, bg = '#009ea0',bd = 0)
    header.grid(row = 0, columnspan=10 , column = 0)

    title = tk.Label(main, text  = 'Papaya \nProject', bd = 0, fg ='#343434'  ,
                font=('sans',9,'bold'), justify = 'left', bg = '#009ea0', pady=5)
    title.grid(row = 0, column=1, columnspan=5)
    title.place(x = 60)
    
    restart = tk.Button(main,text='Restart',command=lambda: restart_buttons(main), padx= 20)
    restart.grid(row = 0, column = 3,columnspan=2,sticky=tk.W)

    stop = tk.Button(main,text='Stop', padx= 20,command=lambda : stop_buttons(main))
    stop.grid(row = 0, column = 6,columnspan=2 ,sticky=tk.W)

    imageframe = tk.Frame (main, width= 244, height= 244, bg = "#343434" )
    imageframe.grid(row = 1, column=0,sticky="nsew", columnspan=3, rowspan =10 )
    
    color_disease = []
    for key in disease_list_final.keys():
        if disease_list_final[key]>95:
            color_disease.append("red")
        else:
            color_disease.append("lightblue")
    print(color_disease)


    output = tk.Label(main,text = 'Output',bg = 'lightblue',fg = 'black',borderwidth=2,relief='raise' , pady=10, font =roboto_font,background='white')
    output.grid(row = 1, columnspan=7, rowspan=2, column= 3, sticky="nsew")
    #output.place(x = 300, y = 280)
    maturity = tk.Label(main,text = "maturity",bg = 'lightblue',fg = 'black',bd = 0 ,borderwidth=2,relief='groove' , pady = 10,font =roboto_font,background='#009ea0')
    maturity.grid(row = 3, columnspan= 3, rowspan=1, column= 3,sticky="nsew")
    maturity_text = tk.Label(main,text = maturity_detected,bg = 'lightblue',fg = 'black',borderwidth=2,relief='groove',bd = 0,pady = 10  , font =roboto_font,background='lightblue')
    maturity_text.grid(row = 3, columnspan= 4, rowspan=1, column= 6,sticky="nsew")
    disease = tk.Label(main,text = 'Diseases',bg = 'lightblue',fg = 'black',bd = 0,borderwidth=2,relief='groove' , pady = 10,font =roboto_font,background='#009ea0')
    disease.grid(row = 4, columnspan=3, rowspan=1, column= 3, sticky="nsew")
    disease_text = tk.Label(main,text = highest,bg = 'lightblue',fg = 'black',bd = 0 ,pady = 10, borderwidth=2,relief='groove',font =roboto_font,background='lightblue')
    disease_text.grid(row = 4, columnspan= 4, rowspan=1, column= 6,sticky="nsew")

    #diseases_frame = tk.Frame(main,bg = "yellow")
    #diseases_frame.grid(row=5,rowspan = 5,column=3,columnspan=7,sticky="NSEW")

    anthracnose = tk.Label(main,text = 'anthracnose',bg = 'lightblue',fg = 'black',bd = 0 , font =roboto_font,background='#009ea0')
    anthracnose.grid(row = 5, columnspan=3, rowspan=1, column= 3,sticky="nsew")
    anthracnose_text = tk.Label(main,text = disease_list_final['anthracnose'],bg = 'lightblue',fg = 'black',bd = 0 , font =roboto_font,background=color_disease[0])
    anthracnose_text.grid(row = 5, columnspan= 4, rowspan=1, column= 6,sticky="nsew")
   
    black_spot = tk.Label(main,text = 'black_spot',bg = 'lightblue',fg = 'black',bd = 0 , font =roboto_font,background='#009ea0')
    black_spot.grid(row = 6, columnspan=3, rowspan=1, column= 3, sticky="nsew")
    black_spot_text = tk.Label(main,text =disease_list_final['black_spot'],bg = 'lightblue',fg = 'black',bd = 0 , font =roboto_font,background=color_disease[1])
    black_spot_text.grid(row = 6, columnspan= 4, rowspan=1, column= 6,sticky="nsew")
   
    phytophthora = tk.Label(main,text = 'phytophthora',bg = 'lightblue',fg = 'black',bd = 0, font =roboto_font,background='#009ea0')
    phytophthora.grid(row = 7, columnspan=3, rowspan=1, column= 3, sticky="nsew")
    phytophthora_text = tk.Label(main,text = disease_list_final['phytophthora'],bg = 'lightblue',fg = 'black',bd = 0 , font =roboto_font,background=color_disease[2])
    phytophthora_text.grid(row = 7, columnspan= 4, rowspan=1, column= 6,sticky="nsew")
   
    powdery_mildew = tk.Label(main,text = 'powdery_mildew',bg = 'lightblue',fg = 'black',bd = 0 , font =roboto_font,background='#009ea0')
    powdery_mildew.grid(row = 8, columnspan=3, rowspan=1, column= 3, sticky="nsew")
    powdery_mildew_text = tk.Label(main,text = disease_list_final['powdery_mildew'],bg = 'lightblue',fg = 'black',bd = 0, font =roboto_font,background=color_disease[3])
    powdery_mildew_text.grid(row = 8, columnspan= 4, rowspan=1, column= 6,sticky="nsew")

    ring_spot = tk.Label(main,text = 'ring_spot',bg = 'lightblue',fg = 'black',bd = 0 , font =roboto_font,background='#009ea0')
    ring_spot.grid(row = 9, columnspan=3, rowspan=1, column= 3, sticky="nsew")
    ring_spot_text = tk.Label(main,text = disease_list_final['ring_spot'],bg = 'lightblue',fg = 'black',bd = 0, font =roboto_font,background=color_disease[4])
    ring_spot_text.grid(row = 9, columnspan= 4, rowspan=1, column= 6,sticky = "nsew")

    gram = tk.Label(main,text = 'Weight',bg = 'lightblue',fg = 'black',bd = 0 ,borderwidth=2,relief='groove',pady = 10,font =roboto_font,background='#009ea0')
    gram.grid(row = 10, columnspan=3, rowspan=1, column= 3, sticky="nsew")
    gram_text = tk.Label(main,text = "1000 grams",bg = 'lightblue',fg = 'black',bd = 0, pady = 10,font =roboto_font,background='lightblue')
    gram_text.grid(row = 10, columnspan= 4, rowspan=1, column= 6,sticky = "nsew")
    """
    maturity_text = tk.Label(main, text ='Maturity Level : ', bg = 'lightblue' ,fg = 'black',pady=10, font =('Times',10))
    maturity_text.grid(row = 2, column= 3 , columnspan= 2, sticky=tk.W )

    

    maturity_bg= tk.Frame(main, width=60, height= 40, bg = '#333333')
    maturity_bg.grid(row = 2, column= 4 , columnspan= 3, sticky=tk.W )
    

    maturity = tk.Label(maturity_bg, text =maturity_detected, bg = '#DDDDDD' ,fg = '#222222',pady=10, font =('Times',10))
    maturity.grid(row = 2, column= 4 , columnspan= 2, sticky=tk.W )

    frame = tk.Frame(main, bg="#f0f0f0", padx=10, pady=10, background='#982312')
    frame.grid(row = 1, column=3, sticky= tk.W, columnspan=3, rowspan =10 )

    # Create a label for each key-value pair in the dictionary
    
    

    size_text = tk.Label(main, text ='Weight : ', bg = 'lightblue' ,fg = 'black',pady=10)
    size_text.grid(row = 4, column= 3 , columnspan= 2, sticky=tk.NW )

    size_bg= tk.Frame(main, width=60, height= 40, bg = '#333333')
    size_bg.grid(row = 4, column= 4 , columnspan= 3, sticky=tk.NW )

    size = tk.Label(size_bg, text =str(weight) + 'grams' , bg = '#DDDDDD' ,fg = '#222222',pady=10, font =('Times',10))
    size.grid(row = 4, column= 4 , columnspan= 2, sticky=tk.NW )
    """

    #body.place(x = 0, y = 50)
    
    footer = tk.Frame(main,width=480, height = 35, bg = 'white',bd = 0)
    footer.grid(row = 15,column=0 ,columnspan= 10, rowspan=3, sticky = tk.N)

    
    
    
    
    #navigate = Frame(s,width=150, height=20,bg = 'blue',bd = 0 )
    #navigate.grid(row = 6, columnspan= 3, column= 0, sticky= NW,padx = 20)
    #navigate.place(x = 65 ,y = 300)
    """
    logo = Image.open(r'papaya_logo.png')
    logo = logo.resize((40,30))
    logo = ImageTk.PhotoImage(logo)
    image_label = tk.Label(main,image=logo, bd = 0,bg = 'lightblue')
    image_label.image = logo
    image_label.grid(row = 0, column = 0,columnspan=4 ,sticky=tk.E)
    image_label.place(x = 5, y = 5)
    """
   
    

    """
    
    path = os.listdir(folder)
    print(path)
    newpath = os.path.join(folder,(path[index1]))
    print(newpath)"""
    
    def update_img(newpath):
        img = Image.open(newpath)

        img = img.resize((244,244))
        img = np.array(img)
        image=Image.fromarray(img)
        img = ImageTk.PhotoImage(image)
        imglabel = tk.Label(imageframe,image = img)
        imglabel.image = img
        imglabel.grid(row = 1, column=0, sticky= "nsew", columnspan=10, rowspan =4)
        #imglabel.place(x= 5, y = 55)
        return (imglabel)
    
    update_img(newpath)

    def rightbutton(newpath,path):
        global index1
        if index1 < len(path) - 1:
            index1 += 1

        print(index1)
        newpath  = os.path.join(folder,(path[index1]))
        update_img(newpath)
        items = tk.Label(main, text = 'Image %i of %i' %(index1+1, len(path)), padx = 10)
        items.grid(row=16, column=1)

    def leftbutton(newpath,path):
        global index1
        
        if index1 > 0:
            index1 -= 1

        print(index1)
        newpath  = os.path.join(folder,(path[index1]))
        
        print(newpath)
        
        update_img(newpath)

        items = tk.Label(main, text = 'Image %i of %i' %(index1+1, len(path)), padx = 10)
        items.grid(row=16, column=1)




    right = tk.Button(main, text = ">", command = lambda : rightbutton(newpath,path),padx = 10 , bg ='#009ea0')
    left = tk.Button(main, text = "<", command = lambda : leftbutton(newpath,path),padx = 10, bg ='#009ea0' )

    right.grid(row = 16, column = 2 ,sticky=tk.NW)
    #right.place(y = 258, x= 410 )
    left.grid(row = 16, column=0, sticky=tk.NE)
    #left.place(y = 258, x= 300 )

    items = tk.Label(main, text = 'Image %i of %i' %(index1+1, len(path)), padx = 10)
    items.grid(row=16, column=1)


    
  
    main.mainloop()

In [11]:
def main(w, progress,l4):
    global Arduino, r, a,cam
    
    #Delete existing directory 'test'
    shutil.rmtree(dirname1)

    #Creates directory 'test' 
    os.mkdir(dirname1)

    #Arduino.write(b'E')
    time.sleep(2)

    for i in range(2):
        c=0
        ret, frame = cam.read()

        #crop
        height, width, _ = frame.shape # get the height and width of the frame
        crop_size = min(height, width) # get the minimum of the two dimensions
        crop_start = (width - crop_size) // 2, (height - crop_size) // 2 # calculate the starting point of the crop
        crop_end = crop_start[0] + crop_size, crop_start[1] + crop_size # calculate the ending point of the crop
        frame = frame[crop_start[1]:crop_end[1], crop_start[0]:crop_end[0]] # crop the frame to a square aspect ratio
        

        #Resize the frame(image captured) to 224x224
        frame = cv2.resize(frame, (224,224))
        #assign fram ti
        disease_img = frame
        frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        

        capture(frame)
        
        frame1 = np.array(frame)
        maturity_img = cv2.cvtColor(frame1,cv2.COLOR_BGR2RGB)

        #confirm_papaya = (detect_papaya(disease_img,model_detect))
        #print('CONFIRM PAPAYA: ', confirm_papaya)
        l4.config(text="image '"+ str(i+1) + "'of 4............",fg='white',bg=a)
        

        time.sleep(0.03)
        r=r+10
        progress['value']=r
        w.update_idletasks()
        
        #roll()
        time.sleep(1)
        

        img_list = []
        """
        Arduino.write(b'H')
        print('High')
        line  = Arduino.readline().decode('utf-8').rstrip()
        
        
        
        print(line)
        while True:
            c += 1
            if line == "Done High" or c>10:
                break
        Arduino.write(b'L')
        time.sleep(2)
        """
    #Arduino.write(b'D')
    #Arduino.close()



In [12]:
def output(img2,model_disease):
    categories = ['anthracnose','black_spot','phytophthora','powdery_mildew','ring_spot','None']
    
    disease_list = detect_disease(img2,model_disease)
    
    #green, yellow = detect_maturity(img1)

    #print("Type: ", type(percentage))
    
    """
    print('###########################')
    for key in disease_list:
       print(key + ' = ' , disease_list[key])
    print('###########################')
    """

    return disease_list

In [13]:
def detect_papaya(image):
    global model_detect
    model = model_detect
    predictions = 'None'
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    #image = Image.open(path).convert('RGB')
    size = (224, 224)
    #image = ImageOps.fit(image, size, Image.ANTIALIAS)
    
    image_array = image
    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
    data[0] = normalized_image_array
    prediction = model.predict(data)
    enumerate_object = enumerate(prediction[0])
    index = 0
    for i in range(len(prediction[0])):
        
        if (prediction[0][i]*100) >= 90:
            if i == 0:
                predictions = False
                index = i
                
            if i == 1:
                predictions = True
                index = i
    print('*************************************************************************')
    print('Papaya Detection Prediction: ', predictions)
    print("PREDICTIONS: ",prediction)
    print('*************************************************************************')
    return predictions


In [14]:

def detect_maturity(img):

    maturity_results = ()
    #Green_HSV = (33,0,0,90,255,255)#0,179,0,255,0,98 #32 104 0 123 7 219
    #Yellow_HSV = (1,0,0,32,255,255)
    #default_HSV = (1,0,0,86,255,255)

    Green_HSV = (38,0,0,90,255,255)#0,179,0,255,0,98 #32 104 0 123 7 219
    Yellow_HSV = (1,0,0,37,255,255)



    #low_HSV = np.array([0,0,0])  # 0 69 0 255 0 169#GREEN 0 179 0 238 0 115 #DEFAULT 25,45,0 ,179,255,255 , # 2 46 16 255 62 202
    #up_HSV = np.array([94,255,255])

    default_HSV = (1,43,0,70,255,255)

    Original_images = process_image(default_HSV,img) #img,imgresult,imgContour,imgCanny,imgDialation,mask
    imgStack = results(Original_images)
    #x = find_area(Original_images[4],Original_images[2])
    #getContours(Original_images[4],Original_images[2])
    imgStack = results(Original_images)

    color1 = 'green'
    Green_area,Green_result,Green_img = findColor(Original_images[1],Green_HSV,color1)

    color2 = 'yellow'
    Yellow_area,Yellow_result,Yellow_img = findColor(Original_images[1],Yellow_HSV,color2)

    maturity_results = img, Original_images[1],Yellow_img ,Green_img
    resultStack = result_stack(maturity_results)

    Original_nonzero = np.nonzero(Original_images[1])
    Original_nonzero = len(Original_nonzero[1])+len(Original_nonzero[2])+len(Original_nonzero[0])
    Green_nonzero = np.nonzero(Green_img)
    Green_nonzero = len(Green_nonzero[1])+len(Green_nonzero[2])+len(Green_nonzero[0])
    Yellow_nonzero = np.nonzero(Yellow_img)
    Yellow_nonzero = len(Yellow_nonzero[1])+len(Yellow_nonzero[2])+len(Yellow_nonzero[0])

    #print("Total Non-zerofor Original pixels = ",Original_nonzero)
    #print("Total Non-zero for %s pixels = %f" %(color1,Green_nonzero))
    #print("Total Non-zerofor %s pixels = %f "%(color2,Yellow_nonzero))
    green=Green_nonzero/Original_nonzero*100
    yellow=Yellow_nonzero/Original_nonzero*100
    #print(color1,' is ',green,' percent of papaya' )
    #print(color2,' is ',yellow,' percent of papaya' )

    return green, yellow


In [15]:
def detect_disease(image,model):
  image = cv2.resize(image, dsize=[224,224])
  cv2.imwrite('Disease 1.png',image)
  data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
  #image = Image.open(path).convert('RGB')
  size = (224, 224)
  #image = ImageOps.fit(image, size, Image.ANTIALIAS)
  
  image_array = image
  
  normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
  data[0] = normalized_image_array
  prediction = model.predict(data)
  print("Prediction: ",prediction)
  #,'No_Disease':None
  disease = {'anthracnose':None,'black_spot':None,'phytophthora':None,'powdery_mildew':None,'ring_spot':None}
  i=0
  for key in disease:
    value =float("{:.2f}".format((prediction[0][i]) * 100))
    

    disease[key] = value
    i+=1
  
  print(disease)



  return disease

In [16]:
cam = cv2.VideoCapture(0)



In [17]:
"""
Arduino = serial.Serial("COM6",57600)
img_counter = 0

while True:
    Arduino.write(b'R')
    line  = Arduino.readline().decode('utf-8').rstrip()
    if line == 'Startup is complete':
        break

"""

'\nArduino = serial.Serial("COM6",57600)\nimg_counter = 0\n\nwhile True:\n    Arduino.write(b\'R\')\n    line  = Arduino.readline().decode(\'utf-8\').rstrip()\n    if line == \'Startup is complete\':\n        break\n\n'

In [18]:
def roll():
    global Arduino
    c=0
    Arduino.write(b'H')
    print('High')
    line  = Arduino.readline().decode('utf-8').rstrip()
    print(line)
    time.sleep(2.5)
    while True:
        c += 1
        if line == "Done High" or c>10:
            break
    Arduino.write(b'L')
    return True
    

In [19]:


def weight(Arduino):

    print('Weight')


    ave = []
    cnt = 0
    while True:
        time.sleep(0.5)
        Arduino.write(b'R')
        if Arduino.in_waiting>0:
            try:
                
                myData = Arduino.readline()
                grams = str(myData).strip("b'").rstrip("\\r\\n'")
                #print(grams)
                ave.append(float(grams))
                cnt += 1
                if cnt >10:
                    break
            except ValueError:
                continue
    return (grams)
        

In [20]:
#Arduino.write(b'E')

In [21]:
dirname1 = r'test'
dirname2 = r'TESTING IMAGES\testing_antrachnose\1'
img_counter = 0
r = 0
index1 = 0
a='#249794'
cam.open(0)
splash()
cam.open(0)
#Arduino.open()

Program is Stoping


True

In [24]:
img = cv2.imread(r'TESTING_IMAGES\testing_antrachnose\1\papaya_images_0.png')
cv2.imshow('img',img)


error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
